In [7]:
import openai
from datetime import datetime

In [11]:
%%timeit -n 10
t = round(datetime.now().timestamp() * 1000)

1.44 µs ± 362 ns per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [18]:
msgs = [dict(role="system", content="What is the average rating of the app?"), dict(role="user", content="What is the average rating of the app?")]

In [15]:
for t2 in t:
    for k, v in t2.items():
        print(k, v)

role system
content What is the average rating of the app?
role user
content What is the average rating of the app?


In [19]:
{msg["role"]: msg["content"] for msg in msgs}

{'system': 'What is the average rating of the app?',
 'user': 'What is the average rating of the app?'}

In [1]:
import sqlmodel

In [8]:
ANALYST_SYSTEM_MSG = """As a skilled analyst specializing in customer review data, your task is to answer questions from bank employees and app developers about the customer review data for either the mobile bank app or the bank itself.

When answering questions you have access to querying review data, analysing queried data and plotting data.
"""

In [9]:
data_func = {
    "name": "get_data",
    "description": "Fetches review data to be analysed. This can either be in the form of raw review text and metadata or review statistics calculated from a sql query.",
    "parameters": {
        "type": "object",
        "properties": {
                "query": {
                    "type": "string",
                    "description": "Natural language query specifying the data to be fetched and/or the statistics to be calculated. For complex queries, divide them into sub queries.",
                },
        },
        "required": ["query"],
    },
}

# plot_func = {}
# analysis_func = {}

In [10]:
prompt = "Hvordan har kunderne anmeldt Danske Bank appen i forhold til SDC appen?"

res = openai.ChatCompletion.create(
    model='gpt-4-0613',  # swap for gpt-3.5-turbo-0613 if needed
    messages=[{"role": "system", "content": ANALYST_SYSTEM_MSG}, {"role": "user", "content": prompt}],
    functions=[data_func]
)

In [11]:
res['choices'][0]

<OpenAIObject at 0x112324950> JSON: {
  "index": 0,
  "message": {
    "role": "assistant",
    "content": null,
    "function_call": {
      "name": "get_data",
      "arguments": "{\n  \"query\": \"Fetch review data for Danske Bank app and SDC app\"\n}"
    }
  },
  "finish_reason": "function_call"
}

In [1]:
import time

In [2]:
start = time.time()

In [3]:
end = time.time()

In [4]:
start

1692256825.8228328

In [5]:
end

1692256831.257429

In [6]:
end - start

5.434596061706543

In [27]:
print(res['choices'][0].message["function_call"]["arguments"])

{
  "source": "app",
  "bank": "Danske Bank",
  "retrieval_method": "statistics"
}


In [22]:
res['choices'][0]

<OpenAIObject at 0x12ab17e90> JSON: {
  "index": 0,
  "message": {
    "role": "assistant",
    "content": null,
    "function_call": {
      "name": "get_data",
      "arguments": "{\n  \"source\": \"app\",\n  \"bank\": \"Danske Bank\",\n  \"retrieval_method\": \"statistics\"\n}"
    }
  },
  "finish_reason": "function_call"
}

In [ ]:
"""
1. source: Mandatory field. Set as 'app' for mobile bank app reviews and 'bank' for general bank reviews.
2. date_range: Include 'start' and 'end' fields in 'yyyy-mm-dd' format when specified in the question, otherwise, omit this field. The current date is {{ current_date }}.
3. category: Set this field to one of the given values when a review type is mentioned, otherwise, omit it: 'other', 'functionality', 'ui', 'error', 'stability', 'customer service', 'counseling', 'mobile/web bank', 'fees and interest rates'.
4. bank: Mention the bank's name if the question is about a specific bank, otherwise, omit this field.
5. country: Use the respective country code (e.g., 'DK', 'NO', 'SE', 'FO', 'UK', 'NZ') if a specific country is mentioned, otherwise, omit this field.
6. similarity_query: Set this field with the query text if a specific semantic query is mentioned, otherwise, omit it.
7. retrieve: Set as 'reviews' to retrieve reviews and 'statistics' to retrieve statistics. If not specified, the default value is 'reviews'.

For complex questions, divide them into sub-questions and refine results using the parameters 'date_range', 'category', 'bank', and 'country', with the optional 'similarity_query' for querying or ordering by semantic similarity. Ensure you accurately transform the question into the JSON structure, taking care to attend to the subtleties and details provided in the prompt."""

"""

In [ ]:
def get_completion_from_messages(messages, 
                                 model="gpt-3.5-turbo", 
                                 temperature=0):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, 
    )
    return response.choices[0].message["content"]


response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=[
       {"role": "user", "content": "Explain how to assemble a PC"}
    ],
    functions=[
        {
          "name": "get_answer_for_user_query",
          "description": "Get user answer in series of steps",
          "parameters": StepByStepAIResponse.schema()
        }
    ],
    function_call={"name": "get_answer_for_user_query"}
)